In [8]:
import numpy as np
import random
from shapely.geometry import Polygon,Point
import pandas as pd 
import h3
import folium

import matplotlib.pyplot as plt

In [9]:
#defining the desired polygon
poly = Polygon([(27.708160, 85.339501),
                (27.703002, 85.372067),
                (27.722503, 85.374131),
                (27.718207, 85.345394)])

In [10]:
#Defining the randomization generator

#for clients
def polygon_random_points_clients (poly, num_points):
    min_x, min_y, max_x, max_y = poly.bounds
    
    points = []

    while len(points) < num_points:
        random_point = Point([random.uniform(min_x, max_x), random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            points.append(random_point)
    return points

#for riders
def polygon_random_points_riders (poly, num_points):
    min_x, min_y, max_x, max_y = poly.bounds
    
    points = []

    while len(points) < num_points:
        random_point = Point([random.triangular(min_x, max_x), random.triangular(min_y, max_y)])
        if (random_point.within(poly)):
            points.append(random_point)
    return points

# Choose the number of points desired. 
points_clients = polygon_random_points_clients(poly,100)
points_riders = polygon_random_points_riders(poly,60)

latitude_clients = []
longitude_clients = []
latitude_riders = []
longitude_riders = []

# Printing the results.
for p in points_clients:
    latitude_clients.append(p.x)
    longitude_clients.append(p.y)

for p in points_riders:
    latitude_riders.append(p.x)
    longitude_riders.append(p.y)

In [11]:
df_clients= pd.DataFrame()
df_riders = pd.DataFrame()

df_clients['Latitude_c'] = latitude_clients
df_clients['Longitude_c'] = longitude_clients

df_riders['Latitude_r'] = latitude_riders
df_riders['Longitude_r'] = longitude_riders
df_clients.head()

,Latitude_c,Longitude_c
0,27.718797,85.365760
1,27.718692,85.362771
2,27.709647,85.367249
3,27.710021,85.357329
4,27.718772,85.371355


In [12]:
df_riders.head()

,Latitude_r,Longitude_r
0,27.707135,85.355041
1,27.714116,85.349745
2,27.715161,85.344856
3,27.708792,85.356291
4,27.714501,85.354337


In [13]:
map1 = folium.Map(location=[df_clients.Latitude_c.mean(), df_clients.Longitude_c.mean()],
                 zoom_start=14, control_scale=True)

In [14]:
for index, location_info in df_clients.iterrows():
    folium.Marker([location_info["Latitude_c"], location_info["Longitude_c"]],icon=folium.Icon(color='green',icon='user', prefix='fa')).add_to(map1)

for index, location_info in df_riders.iterrows():
    folium.Marker([location_info["Latitude_r"], location_info["Longitude_r"]],icon=folium.Icon(color='red',icon='motorcycle', prefix='fa')).add_to(map1)
    
map1

In [15]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

In [27]:
kms_per_radian = 6371.0088
epsilon = 1/ kms_per_radian
clustering = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(np.radians(df_clients))
cluster_labels = clustering.labels_
num_clusters = len(set(cluster_labels))
num_clusters

1

In [28]:
np.radians(df_clients)

,Latitude_c,Longitude_c
0,0.483784,1.489914
1,0.483782,1.489861
2,0.483625,1.489940
3,0.483631,1.489766
4,0.483784,1.490011
...,...,...
95,0.483655,1.489533
96,0.483726,1.489682
97,0.483731,1.489772
98,0.483735,1.489961


In [29]:
cluster_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [30]:
silhouette_score(df_clients, cluster_labels, metric='euclidean', sample_size=None, random_state=None)

ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)